# Weaviate 1.33 enablement session

This notebook demonstrates all the major features introduced in Weaviate v1.33, including compression by default, advanced quantization, server-side batching, and more.

## Table of Contents
- [0. Setup and prerequisites](#setup)
- [1. Server-side batch imports (Preview)](#server-batch)
- [2. Collection aliases (GA)](#aliases)
- [3. Compression by default & 8-bit RQ (GA)](#compression-default)
- [4. 1-bit Rotational Quantization (Preview)](#1bit-rq)
- [5. New filter operators: `ContainsNone` & `Not`](#filters)
- [6. OIDC Group management](#oidc-groups)

<div class="alert alert-info">
<b>Warning:</b>
<ul>
  <li>The documentation and blog post links are pointing to draft versions as the materials haven't been officially published yet.</li>
  <li>The Docker Compose files use Weaviate RC (Release Candidate versions) and not official builds.</li>
  <li>The Weaviate Python client library is also under development for v1.33.</li>
</ul>
</div>

<a id="setup"></a>
## Setup and prerequisites

Make sure you have the following services up and running:
- `docker-compose-prerelease-1-33.yml` up and running (for steps 1-5)
- `docker-compose-prerelease-1-33-keycloak.yml` up and running (for step 6)

Install the 1.33 dev Weaviate Python client:

```bash
!pip install git+https://github.com/weaviate/weaviate-python-client.git@feat/server-side-batching # For step 1
!pip install weaviate-client==4.17.0-rc0 # For steps 2-6
```

Connect to a local Weaviate instance:

In [2]:
import os
import weaviate
from dotenv import load_dotenv
from pprint import pprint
from weaviate.classes.config import Configure, DataType, Property, VectorDistances
from weaviate.collections.classes.filters import Filter


# Load environment variables from .env file
load_dotenv()

# Connect to Weaviate instance
client = weaviate.connect_to_local(
    headers={
        "X-Openai-Api-Key": os.environ[
            "OPENAI_API_KEY"
        ]  # Replace with your inference API key
    }
)

# Check connection
print(f"Connected to Weaviate: {client.is_ready()}")
print(f"Weaviate version: {client.get_meta()['version']}")

Connected to Weaviate: True
Weaviate version: 1.33.0-rc.1


<a id="server-batch"></a>
## 1. Server-side batch imports

Server-side batching (automatic batching) lets the server optimize data ingestion for maximum performance.

### ⚠️ Important considerations
- **Preview feature** - Not recommended for production

### Key advantages
- ✅ **No manual tuning** - Server optimizes batch size automatically
- ✅ **Dynamic backpressure** - Prevents server overload
- ✅ **Asynchronous error handling** - Errors don't interrupt flow

### Docs
- [Concepts: Data import](https://v1-33-main--docs-weaviate-io.netlify.app/weaviate/concepts/data-import)
- [How-to: Batch import](https://v1-33-main--docs-weaviate-io.netlify.app/weaviate/manage-objects/import#server-side-batching)
- [Tutorial: Batch data import](https://v1-33-main--docs-weaviate-io.netlify.app/weaviate/tutorials/import)

In [2]:
def create_collection(name):
    client.collections.delete(name)  # Clean up previous test
    # Create test collection
    test_collection = client.collections.create(
        name=name,
        properties=[
            Property(name="title", data_type=DataType.TEXT),
            Property(name="content", data_type=DataType.TEXT),
        ]
    )

# Example usage
large_dataset = [
    {"title": f"Article {i}", "content": f"Content for article {i}..."} 
    for i in range(1000)
]

In [3]:
# Traditional client-side batching (old way)
def traditional_batching(collection_name, data_objects):
    """Manual batching - requires tuning batch_size and concurrent_requests"""
    collection = client.collections.get(collection_name)
    
    # Manual configuration required
    with collection.batch.fixed_size(
        batch_size=100,  # Manual tuning needed
        concurrent_requests=2  # Manual tuning needed
    ) as batch:
        for obj in data_objects:
            batch.add_object(properties=obj)
    
    print(f"Imported {len(collection)} objects with manual batching")

create_collection("BatchImportTest")
traditional_batching("BatchImportTest", large_dataset)


Imported 1000 objects with manual batching


In [ ]:
# NEW: Server-side automatic batching (v1.33)
def automatic_batching(collection_name, data_objects):
    """Server-side batching - automatic optimization!"""
    collection = client.collections.get(collection_name)
    
    # Server automatically optimizes everything!
    with collection.batch.automatic() as batch:
        for obj in data_objects:
            batch.add_object(properties=obj)
            
    print(f"✅ Imported {len(data_objects)} objects with automatic batching")

automatic_batching("BatchImportTest", large_dataset*10)

# TODO: Check how concurrent_requests are handled

Scaling up sending: 1000 -> 4000
Waiting for background thread to finish...


✅ Imported 10000 objects with automatic batching


<a id="aliases"></a>
## 2. Collection aliases (GA)

Collection aliases enable zero-downtime migrations and flexible production deployments.

### ⚠️ Important considerations
- Don't forget to install the client version: `!pip install weaviate-client==4.17.0-rc0`

### Key benefits
- ✅ **Zero-downtime migrations** - Switch collections instantly
- ✅ **Collection definition changes** - Otherwise immutable parameters can be changed this way
- ✅ **Generally available** - Production-ready!

### Docs
- [How-to: Collection aliases](https://v1-33-main--docs-weaviate-io.netlify.app/weaviate/manage-collections/collection-aliases)
- [Reference: Collection definition](https://v1-33-main--docs-weaviate-io.netlify.app/weaviate/config-refs/collections#collection-aliases)
- [Tutorial: Zero-downtime collection migration with aliases](https://v1-33-main--docs-weaviate-io.netlify.app/weaviate/tutorials/collection-aliases)

In [3]:
# Scenario: Migrating a production collection with zero downtime

# Cleanup
client.collections.delete("Products_v1")  # Clean up previous test
client.collections.delete("Products_v2")  # Clean up previous test
client.alias.delete(alias_name="ProductsAlias")  # Clean up previous alias

# Step 1: Create initial production collection
def setup_production_collection():
    """Create initial production collection"""
    v1_collection = client.collections.create(
        name="Products_v1",
        properties=[
            Property(name="name", data_type=DataType.TEXT),
            Property(name="description", data_type=DataType.TEXT),
            Property(name="price", data_type=DataType.NUMBER),
            Property(name="category", data_type=DataType.TEXT),
        ],
        vector_config=Configure.Vectors.text2vec_openai()
    )
    
    # Import production data
    products = [
        {"name": "Laptop Pro", "description": "High-performance laptop", "price": 1299.99, "category": "electronics"},
        {"name": "Wireless Mouse", "description": "Ergonomic wireless mouse", "price": 29.99, "category": "accessories"},
        {"name": "USB-C Hub", "description": "7-in-1 USB-C hub", "price": 49.99, "category": "accessories"},
    ]
    
    with v1_collection.batch.fixed_size() as batch:
        for product in products:
            batch.add_object(properties=product)
    
    print("✅ Production collection 'Products_v1' created and populated")
    return v1_collection

setup_production_collection()

✅ Production collection 'Products_v1' created and populated


In [4]:
# Step 2: Create alias pointing to production
def create_production_alias():
    """Create alias for production use"""
    client.alias.create(
        alias_name="ProductsAlias",  # This is what applications use
        target_collection="Products_v1"
    )
    print("✅ Alias 'ProductsAlias' -> 'Products_v1' created")

create_production_alias()

✅ Alias 'ProductsAlias' -> 'Products_v1' created


In [8]:
# Step 3: Applications use the alias (not the versioned name)
def application_query():
    """Simulate application querying via alias"""
    # Application always uses the alias, not version-specific names
    products = client.collections.use("ProductsAlias")  # Using alias!
    
    results = products.query.near_text(
        query="laptop accessories",
        limit=3
    )
    
    print("\n🔍 Query results via alias 'ProductsAlias':")
    for item in results.objects:
        print(f"  - {item.properties['name']}: ${item.properties['price']}")

application_query()


🔍 Query results via alias 'ProductsAlias':
  - Laptop Pro 2: $1299.99
  - Wireless Mouse 2: $29.99
  - USB-C Hub 2: $49.99



![Collection alias usage](./img/collection_alias_tutorial.png)

In [6]:
client.collections.delete("Products_v2")  # Clean up previous test
# Step 4: Prepare new version with improvements
def prepare_v2_collection():
    """Create improved v2 collection"""
    v2_collection = client.collections.create(
        name="Products_v2",
        properties=[
            Property(name="name", data_type=DataType.TEXT),
            Property(name="description", data_type=DataType.TEXT),
            Property(name="price", data_type=DataType.NUMBER),
            Property(name="category", data_type=DataType.TEXT),
            Property(name="brand", data_type=DataType.TEXT),  # NEW field
            Property(name="in_stock", data_type=DataType.BOOL),  # NEW field
        ],
        vector_config=Configure.Vectors.text2vec_openai(
            vector_index_config=Configure.VectorIndex.hnsw(
                distance_metric=VectorDistances.COSINE,
                ef=256,  # Improved search quality
                quantizer=Configure.VectorIndex.Quantizer.rq(),  # Using new default RQ
            )
        ),
    )

    # Migrate and enhance data
    enhanced_products = [
        {
            "name": "Laptop Pro 2",
            "description": "High-performance laptop",
            "price": 1299.99,
            "category": "electronics",
            "brand": "TechCorp",
            "in_stock": True,
        },
        {
            "name": "Wireless Mouse 2",
            "description": "Ergonomic wireless mouse",
            "price": 29.99,
            "category": "accessories",
            "brand": "Logitech",
            "in_stock": True,
        },
        {
            "name": "USB-C Hub 2",
            "description": "7-in-1 USB-C hub",
            "price": 49.99,
            "category": "accessories",
            "brand": "Anker",
            "in_stock": False,
        },
    ]

    with v2_collection.batch.fixed_size() as batch:
        for product in enhanced_products:
            batch.add_object(properties=product)

    print("\n✅ Version 2 collection 'Products_v2' ready.")


prepare_v2_collection()


✅ Version 2 collection 'Products_v2' ready.


In [7]:
# Step 5: Zero-downtime migration
def perform_zero_downtime_migration():
    """Switch alias to new version with zero downtime"""
    print("\n🔄 Performing zero-downtime migration...")
    
    # Update alias to point to v2
    client.alias.update(
        alias_name="ProductsAlias",
        new_target_collection="Products_v2"
    )

    print("✅ Migration complete! 'ProductsAlias' now points to 'Products_v2'")

perform_zero_downtime_migration()


🔄 Performing zero-downtime migration...
✅ Migration complete! 'ProductsAlias' now points to 'Products_v2'


In [9]:
# Step 6: Rollback if needed
def rollback_if_needed():
    """Emergency rollback to previous version"""
    print("\n⚠️ Issue detected - performing rollback...")

    client.alias.update(alias_name="ProductsAlias", new_target_collection="Products_v1")

    print("✅ Rolled back to 'Products_v1'")
    print("🔧 Fix issues in v2 and try again")


# List all aliases
def list_all_aliases():
    """Display all configured aliases"""
    aliases = client.alias.list_all()
    print("\n📋 Configured Aliases:")
    for alias in aliases:
        print(f"Alias: {alias}")


client.alias.create(alias_name="TmpAlias3", target_collection="Products_v2")
list_all_aliases()


📋 Configured Aliases:
Alias: TmpAlias2
Alias: TmpAlias3
Alias: ProductsAlias
Alias: TmpAlias


<a id="compression-default"></a>
## 3. Compression by default & 8-bit RQ (GA)

Starting with v1.33, **8-bit Rotational Quantization (RQ) is enabled by default** for all new collections. This provides up to 4x memory compression while maintaining 98-99% recall.

### Key benefits
- ✅ **Automatic optimization** - No configuration needed by users
- ✅ **Up-to 4x memory compression** - Significant resource savings
- ✅ **98-99% recall maintained** - Minimal accuracy loss with internal dataset testing
- ✅ **No training phase** - Works immediately at index creation
- ✅ **Generally Available (GA)** - Production-ready

### Docs
- [How-to: Rotational Quantization (RQ)](https://v1-33-main--docs-weaviate-io.netlify.app/weaviate/configuration/compression/rq-compression)
- [Concepts: Rotational Quantization (RQ)](https://v1-33-main--docs-weaviate-io.netlify.app/weaviate/concepts/vector-quantization#rotational-quantization)
- [Environment variable: DEFAULT_QUANTIZATION](https://v1-33-main--docs-weaviate-io.netlify.app/deploy/configuration/env-vars#DEFAULT_QUANTIZATION)

In [11]:
client.collections.delete("Articles_Compressed_Default")  # Clean up previous test
# Create a collection - RQ compression is now enabled by default!
default_compressed_collection = client.collections.create(
    name="Articles_Compressed_Default",
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="content", data_type=DataType.TEXT),
        Property(name="author", data_type=DataType.TEXT),
    ],
    vector_config=Configure.Vectors.text2vec_openai()
)

print("✅ Collection created with default RQ compression enabled!")

# Verify compression settings
collection_config = client.collections.get("Articles_Compressed_Default").config.get()
pprint(f"Quantization config: {collection_config.vector_config}")

✅ Collection created with default RQ compression enabled!
("Quantization config: {'default': "
 '_NamedVectorConfig(vectorizer=_NamedVectorizerConfig(vectorizer=<Vectorizers.TEXT2VEC_OPENAI: '
 "'text2vec-openai'>, model={'baseURL': 'https://api.openai.com', 'isAzure': "
 "False, 'model': 'text-embedding-3-small', 'vectorizeClassName': True}, "
 'source_properties=None), '
 'vector_index_config=_VectorIndexConfigHNSW(multi_vector=None, '
 'quantizer=_RQConfig(bits=8, rescore_limit=20), cleanup_interval_seconds=300, '
 "distance_metric=<VectorDistances.COSINE: 'cosine'>, dynamic_ef_min=100, "
 'dynamic_ef_max=500, dynamic_ef_factor=8, ef=-1, ef_construction=128, '
 "filter_strategy=<VectorFilterStrategy.SWEEPING: 'sweeping'>, "
 'flat_search_cutoff=40000, max_connections=32, skip=False, '
 'vector_cache_max_objects=1000000000000))}')


### Change default compression technique

In [12]:
# To disable compression by default (set in environment before starting Weaviate)
# DEFAULT_QUANTIZATION=none

# To use PQ instead of RQ as default
# DEFAULT_QUANTIZATION=pq

client.collections.delete("Articles_Uncompressed")  # Clean up previous test
# Example: Creating a collection without compression (override default)
uncompressed_collection = client.collections.create(
    name="Articles_Uncompressed",
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="content", data_type=DataType.TEXT),
    ],
    vector_config=Configure.Vectors.text2vec_openai(
        vector_index_config=Configure.VectorIndex.hnsw(
            quantizer=Configure.VectorIndex.Quantizer.none()  # Explicitly disable quantization
        )
    ),
)

print("✅ Collection created without compression (override default)")

# Verify compression settings
collection_config = client.collections.get("Articles_Uncompressed").config.get()
pprint(f"Quantization config: {collection_config.vector_config}")

✅ Collection created without compression (override default)
("Quantization config: {'default': "
 '_NamedVectorConfig(vectorizer=_NamedVectorizerConfig(vectorizer=<Vectorizers.TEXT2VEC_OPENAI: '
 "'text2vec-openai'>, model={'baseURL': 'https://api.openai.com', 'isAzure': "
 "False, 'model': 'text-embedding-3-small', 'vectorizeClassName': True}, "
 'source_properties=None), '
 'vector_index_config=_VectorIndexConfigHNSW(multi_vector=None, '
 'quantizer=None, cleanup_interval_seconds=300, '
 "distance_metric=<VectorDistances.COSINE: 'cosine'>, dynamic_ef_min=100, "
 'dynamic_ef_max=500, dynamic_ef_factor=8, ef=-1, ef_construction=128, '
 "filter_strategy=<VectorFilterStrategy.SWEEPING: 'sweeping'>, "
 'flat_search_cutoff=40000, max_connections=32, skip=False, '
 'vector_cache_max_objects=1000000000000))}')


<a id="1bit-rq"></a>
## 4. 1-bit Rotational Quantization

**1-bit RQ** is a cutting-edge preview feature offering extreme compression (up to 32x) while maintaining reasonable recall.

### ⚠️ Important considerations
- **Preview feature** - Not recommended for production

### Key benefits
- ✅ **Up-to 32x memory compression** - Significant resource savings
- ✅ **Good alternative to BQ** - More robust on challenging datasets, throughput decrease compared to BQ, but offers better recall on some datasets

### Docs
- [How-to: Rotational Quantization (RQ)](https://v1-33-main--docs-weaviate-io.netlify.app/weaviate/configuration/compression/rq-compression)
- [Concepts: Rotational Quantization (RQ)](https://v1-33-main--docs-weaviate-io.netlify.app/weaviate/concepts/vector-quantization#rotational-quantization)

In [13]:
client.collections.delete("Articles_1bit_RQ_Preview")  # Clean up previous test
# Create a collection with 1-bit RQ (PREVIEW - NOT FOR PRODUCTION)
one_bit_collection = client.collections.create(
    name="Articles_1bit_RQ_Preview",
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="content", data_type=DataType.TEXT),
    ],
    vector_config=Configure.Vectors.text2vec_openai(
        vector_index_config=Configure.VectorIndex.hnsw(
            quantizer=Configure.VectorIndex.Quantizer.rq(
                bits=1,  # 1-bit quantization
            )
        )
    ),
)

print("⚠️ Collection created with 1-bit RQ (Preview feature)")

# Verify compression settings
collection_config = client.collections.get("Articles_1bit_RQ_Preview").config.get()
pprint(f"Quantization config: {collection_config.vector_config}")

⚠️ Collection created with 1-bit RQ (Preview feature)
("Quantization config: {'default': "
 '_NamedVectorConfig(vectorizer=_NamedVectorizerConfig(vectorizer=<Vectorizers.TEXT2VEC_OPENAI: '
 "'text2vec-openai'>, model={'baseURL': 'https://api.openai.com', 'isAzure': "
 "False, 'model': 'text-embedding-3-small', 'vectorizeClassName': True}, "
 'source_properties=None), '
 'vector_index_config=_VectorIndexConfigHNSW(multi_vector=None, '
 'quantizer=_RQConfig(bits=1, rescore_limit=512), '
 'cleanup_interval_seconds=300, distance_metric=<VectorDistances.COSINE: '
 "'cosine'>, dynamic_ef_min=100, dynamic_ef_max=500, dynamic_ef_factor=8, "
 'ef=-1, ef_construction=128, filter_strategy=<VectorFilterStrategy.SWEEPING: '
 "'sweeping'>, flat_search_cutoff=40000, max_connections=32, skip=False, "
 'vector_cache_max_objects=1000000000000))}')


<a id="filters"></a>
## 5. New filter operators: `ContainsNone` and `Not`

Enhanced filtering capabilities with two new operators:

- **`ContainsNone` Operator**
    Returns objects where the property contains **none** of the specified values.

- **`Not` Operator**
    Provides logical negation of conditions.

In [14]:
client.collections.delete("Articles")  # Clean up previous test
# Create collection
collection = client.collections.create(
    name="Articles",
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="tags", data_type=DataType.TEXT_ARRAY),
        Property(name="category", data_type=DataType.TEXT),
        Property(name="views", data_type=DataType.NUMBER),
    ]
)

# Add 5 sample articles
articles = [
    {"title": "AI Revolution", "tags": ["tech", "ai"], "category": "technology", "views": 1500},
    {"title": "World Cup Final", "tags": ["sports", "soccer"], "category": "sports", "views": 5000},
    {"title": "Healthy Recipes", "tags": ["food", "health"], "category": "lifestyle", "views": 800},
    {"title": "Stock Market Update", "tags": ["finance", "news"], "category": "business", "views": 2000},
    {"title": "Python Tutorial", "tags": ["tech", "programming"], "category": "education", "views": 300},
]

with collection.batch.fixed_size() as batch:
    for article in articles:
        batch.add_object(properties=article)

Now, let's query the data:

In [15]:
# Query with new operators
collection = client.collections.get("Articles")

# Not: Find articles NOT in sports category with 1000+ views
print("\n=== Not operator example ===")
results = collection.query.fetch_objects(
    filters=Filter.all_of(
        [
            Filter.not_(Filter.by_property("category").equal("sports")),
            Filter.by_property("views").greater_than(1000),
        ]
    ),
    limit=5,
)
for obj in results.objects:
    print(f"- {obj.properties['title']}: {obj.properties['views']} views")

# ContainsNone: Find articles WITHOUT sports or news tags
# print("=== ContainsNone operator example ===")
# results = collection.query.fetch_objects(
#     filters=Filter.by_property("tags").contains_none(["sports", "news"]), limit=5
# )
# for obj in results.objects:
#     print(f"- {obj.properties['title']}: {obj.properties['tags']}")



=== Not operator example ===
- AI Revolution: 1500.0 views
- Stock Market Update: 2000.0 views


In [16]:
client.close()

<a id="oidc-groups"></a>
## 6. OIDC Group management

Manage permissions at scale using groups from your identity provider (Keycloak, Okta, Auth0, etc.).

### ⚠️ Important considerations
- **Set up Keycloak for OIDC** - Automatic Keycloak [setup script](https://github.com/weaviate/docs/blob/44a1f7671e8cd6d0aa9da5226b1e274050b5dbf7/_includes/code/python/keycloak_helper_script.py)


### Key features
- ✅ **Automatic permission inheritance** - Users get permissions from their groups
- ✅ **Centralized management** - Define groups in your IdP
- ✅ **Bulk operations** - Assign roles to entire groups
- ✅ **Sync with organization structure** - Permissions match your org chart

### Docs
- [How-to: Manage OIDC groups](https://v1-33-main--docs-weaviate-io.netlify.app/weaviate/configuration/rbac/manage-groups)

In [17]:
import weaviate
from weaviate.classes.init import Auth

# Assign the client to a variable
oidc_client = weaviate.connect_to_local(
    host="localhost",
    port=8580,
    grpc_port=50551,
    auth_credentials=Auth.api_key("root-user-key"),
)

# Use the correct variable name here
print(f"Connected to Weaviate: {oidc_client.is_ready()}")
print(f"Weaviate version: {oidc_client.get_meta()['version']}")

Connected to Weaviate: True
Weaviate version: 1.33.0-rc.1


In [18]:
from weaviate.classes.rbac import Permissions, RoleScope


# Define roles for different teams
def setup_oidc_group_permissions():
    """Configure OIDC group-based permissions"""
    # Cleanup
    oidc_client.roles.delete(role_name="administrator")
    oidc_client.roles.delete(role_name="developer")
    oidc_client.roles.delete(role_name="data_manager")

    permissions = [
        Permissions.roles(
            role="testRole*",  # Applies to all roles starting with "testRole"
            scope=RoleScope.MATCH,  # Only allow role management with the current user's permission level
            # scope=RoleScope.ALL   # Allow role management with all permissions
            create=True,  # Allow creating roles
            read=True,  # Allow reading roles
            update=True,  # Allow updating roles
            delete=True,  # Allow deleting roles
        )
    ]

    oidc_client.roles.create(role_name="administrator", permissions=permissions)
    oidc_client.roles.create(role_name="developer", permissions=permissions)
    oidc_client.roles.create(role_name="data_manager", permissions=permissions)

    # 1. Assign roles to admin group
    oidc_client.groups.oidc.assign_roles(group_id="/admin-group", role_names=["admin"])
    print("✅ Admin group configured with full permissions")

    # 2. Configure engineering team
    oidc_client.groups.oidc.assign_roles(
        group_id="/engineering", role_names=["developer", "data_manager"]
    )
    print("✅ Engineering team configured")


# List all roles for a group
def list_group_permissions(group_id):
    """Display all roles assigned to a group"""
    group_roles = oidc_client.groups.oidc.get_assigned_roles(
        group_id=group_id, include_permissions=True  # Include detailed permissions
    )

    print(f"\n📋 Roles for group '{group_id}':")
    pprint(group_roles)

setup_oidc_group_permissions()
list_group_permissions("/engineering")

✅ Admin group configured with full permissions
✅ Engineering team configured

📋 Roles for group '/engineering':
{'data_manager': Role(name='data_manager',
                      alias_permissions=[],
                      cluster_permissions=[],
                      collections_permissions=[],
                      data_permissions=[],
                      roles_permissions=[RolesPermissionOutput(actions={<RolesAction.UPDATE: 'update_roles'>, <RolesAction.READ: 'read_roles'>, <RolesAction.CREATE: 'create_roles'>, <RolesAction.DELETE: 'delete_roles'>}, role='testRole*', scope='match')],
                      users_permissions=[],
                      backups_permissions=[],
                      nodes_permissions=[],
                      tenants_permissions=[],
                      replicate_permissions=[],
                      groups_permissions=[]),
 'developer': Role(name='developer',
                   alias_permissions=[],
                   cluster_permissions=[],
           

In [19]:

# Revoke specific roles
def update_group_permissions(group_id, roles_to_revoke):
    """Remove specific roles from a group"""
    oidc_client.groups.oidc.revoke_roles(group_id=group_id, role_names=roles_to_revoke)
    print(f"✅ Revoked roles {roles_to_revoke} from {group_id}")

update_group_permissions("/engineering", ["data_manager"])
list_group_permissions("/engineering")

✅ Revoked roles ['data_manager'] from /engineering

📋 Roles for group '/engineering':
{'developer': Role(name='developer',
                   alias_permissions=[],
                   cluster_permissions=[],
                   collections_permissions=[],
                   data_permissions=[],
                   roles_permissions=[RolesPermissionOutput(actions={<RolesAction.UPDATE: 'update_roles'>, <RolesAction.READ: 'read_roles'>, <RolesAction.CREATE: 'create_roles'>, <RolesAction.DELETE: 'delete_roles'>}, role='testRole*', scope='match')],
                   users_permissions=[],
                   backups_permissions=[],
                   nodes_permissions=[],
                   tenants_permissions=[],
                   replicate_permissions=[],
                   groups_permissions=[])}


In [20]:
oidc_client.close()

## Further resources

For more information about the release, check out the [`v1.33` release highlights blog post](https://v1-33-blog--tangerine-buttercream-20c32f.netlify.app/blog/weaviate-1-33-release).